# Amazon Nova Customization SDK - Quick Start Guide

This notebook provides a basic walkthrough of the Amazon Nova Customization SDK for fine-tuning Nova models.

## What You'll Learn

1. Loading and preparing datasets
2. Fine-tuning a Nova model with SFT (Supervised Fine-Tuning)
3. Monitoring training progress
4. Deploying your model

## Prerequisites

- AWS credentials configured
- S3 bucket for data and model artifacts
- IAM permissions for SageMaker and Bedrock
- Nova Customization SDK installed per [its README](https://github.com/aws-samples/sample-nova-customization-sdk?tab=readme-ov-file#installation)
- (If using the SMHP runtime below) Correct version of Sagemaker HyperPod CLI installed; see the SDK README for details

## Step 1: Import Required Modules

In [ ]:
!cd ../ && pip install .

In [ ]:
import os

import boto3
from botocore.exceptions import ClientError, NoCredentialsError, ProfileNotFound


def load_credentials(profile=None):
    """
    Load AWS credentials with fallback behavior.

    Args:
        profile (str, optional): AWS profile name. If provided, loads from credentials file.
                               If None, uses current authenticated AWS session.

    Returns:
        dict: Dictionary containing AWS credentials and region

    Raises:
        RuntimeError: If credential loading fails
    """
    if profile:
        # Try loading from credentials file
        try:
            session = boto3.Session(profile_name=profile)
            credentials = session.get_credentials()

            if not credentials:
                raise RuntimeError(f"No credentials found for profile '{profile}'")

        except ProfileNotFound:
            raise RuntimeError(f"Profile '{profile}' not found in credentials file")
        except Exception as e:
            raise RuntimeError(f"Failed to load credentials from file: {e}")

    else:
        # Try loading from current authenticated session
        try:
            session = boto3.Session()
            credentials = session.get_credentials()

            if not credentials:
                raise RuntimeError("No credentials found in current AWS session")

        except NoCredentialsError:
            raise RuntimeError("No AWS credentials configured")
        except Exception as e:
            raise RuntimeError(f"Failed to load credentials from current session: {e}")

        # Validate credentials by making a test call
    try:
        sts_client = session.client("sts")
        sts_client.get_caller_identity()
    except ClientError as e:
        raise RuntimeError(f"Invalid AWS credentials: {e}")
    except Exception as e:
        raise RuntimeError(f"Failed to validate credentials: {e}")

    return {
        "aws_access_key_id": credentials.access_key,
        "aws_secret_access_key": credentials.secret_key,
        "aws_session_token": credentials.token,
        "region_name": session.region_name or "us-east-1",
    }

In [ ]:
creds = load_credentials()

In [ ]:
# Core imports
from amzn_nova_customization_sdk.dataset.dataset_loader import (
    CSVDatasetLoader,
    JSONDatasetLoader,
    JSONLDatasetLoader,
)
from amzn_nova_customization_sdk.manager.runtime_manager import (
    SMHPRuntimeManager,
    SMTJRuntimeManager,
)
from amzn_nova_customization_sdk.model.model_enums import (
    DeployPlatform,
    Model,
    TrainingMethod,
)
from amzn_nova_customization_sdk.model.nova_model_customizer import NovaModelCustomizer

print("✅ SDK imported successfully!")

## Step 2: Configure Your AWS Resources

In [ ]:
# TODO: Update these values for your environment
S3_BUCKET = "nova-customization-beta"  # TODO: Replace with your S3 bucket
S3_DATA_PATH = f"s3://{S3_BUCKET}/demo/input"
S3_OUTPUT_PATH = f"s3://{S3_BUCKET}/demo/output"

print(f"Data Path: {S3_DATA_PATH}")
print(f"Output Path: {S3_OUTPUT_PATH}")

## Step 3: Prepare Your Dataset

The SDK supports three formats: **JSONL**, **JSON**, and **CSV**. This example uses JSONL.

In [ ]:
# Create sample training data
import json

sample_data = [
    {
        "question": "What is machine learning?",
        "answer": "Machine learning is a subset of artificial intelligence that enables systems to learn and improve from experience without being explicitly programmed.",
    },
    {
        "question": "Explain what AWS is.",
        "answer": "AWS (Amazon Web Services) is a comprehensive cloud computing platform that provides on-demand computing resources and services.",
    },
    {
        "question": "What is Python used for?",
        "answer": "Python is a versatile programming language used for web development, data analysis, artificial intelligence, scientific computing, and automation.",
    },
] * 100

# Save sample data locally
with open("training_data.jsonl", "w") as f:
    for item in sample_data:
        f.write(json.dumps(item) + "\n")

print("✅ Sample data created: training_data.jsonl")

### Load and Transform the Dataset

In [ ]:
# Initialize dataset loader
loader = JSONLDatasetLoader(
    question="question",  # Column name for questions in your data
    answer="answer",  # Column name for answers in your data
)

# Load the data
loader.load("training_data.jsonl")

# Preview the data
print("\n📊 Dataset Preview:")
loader.show(n=3)

In [ ]:
# Transform data for Nova model training
loader.transform(method=TrainingMethod.SFT_LORA, model=Model.NOVA_LITE_2)

print("✅ Data transformed to Converse format")
print("\n📊 Transformed Data Preview:")

loader.show(n=5)

### Split and Save Dataset

In [ ]:
# Split into train/validation sets
train_loader, val_loader, _ = loader.split_data(
    train_ratio=0.7, val_ratio=0.2, test_ratio=0.1
)

# Save datasets
# For production, upload to S3:
train_path = train_loader.save_data(f"{S3_DATA_PATH}/train.jsonl")
val_path = val_loader.save_data(f"{S3_DATA_PATH}/val.jsonl")

print(f"\n✅ Training data saved to: {train_path}")
print(f"✅ Validation data saved to: {val_path}")

## Step 4: Configure Runtime Infrastructure

Choose between:
- **SMTJRuntimeManager**: For SageMaker Training Jobs
- **SMHPRuntimeManager**: For SageMaker HyperPod clusters

In [ ]:
from amzn_nova_customization_sdk.model.model_enums import Platform

In [ ]:
# Option 1: SageMaker Training Jobs (SMTJ)
runtime = SMTJRuntimeManager(
    instance_type="ml.p5.48xlarge",  # Choose appropriate instance
    instance_count=4,  # Number of instances
    # execution_role="<your execution role>",  # TODO: Choose execution role (if different from current role)
)

platform = Platform.SMTJ

print("✅ Runtime configured for SageMaker Training Jobs")
print(f"   Instance Type: {runtime.instance_type}")
print(f"   Instance Count: {runtime.instance_count}")

In [ ]:
# Option 2: SageMaker HyperPod (if using HyperPod cluster)
# Uncomment and configure if using HyperPod:

# runtime = SMHPRuntimeManager(
#     instance_type="ml.p5.48xlarge",
#     instance_count=4,
#     cluster_name="your-cluster-name",
#     namespace="your-namespace"
# )
#
# platform = Platform.SMHP
#
# print("✅ Runtime configured for SageMaker HyperPod")

## Step 5: Initialize Nova Model Customizer

In [ ]:
# Create customizer
customizer = NovaModelCustomizer(
    model=Model.NOVA_LITE_2,  # Choose your Nova model
    method=TrainingMethod.SFT_LORA,  # Training method
    infra=runtime,  # Runtime configuration
    data_s3_path=train_path,  # Training data path
    output_s3_path=S3_OUTPUT_PATH,  # Output path for artifacts
)
print("✅ NovaModelCustomizer initialized")
print(f"   Model: Nova Lite 2.0")
print(f"   Method: SFT with LoRA")

## Step 6: Start Training

In [ ]:
# Define training hyperparameters
training_config = {
    "max_epochs": 3,  # Number of epochs
    "lr": 5e-6,  # Learning rate
    "warmup_steps": 100,  # Warmup steps
    "global_batch_size": 64,  # Batch size
    "max_length": 8192,  # Max sequence length
}

# Start training
training_result = customizer.train(
    job_name="nova-quickstart-training-nova-2", overrides=training_config
)

print("\n🚀 Training job started!")
print(training_result)
print(
    f"   📍 Checkpoint URI where the model will be saved: {training_result.model_artifacts.checkpoint_s3_path}"
)
print(f"   🆔 Job ID: {training_result.job_id}")
print(f"   📂 Output Path: {training_result.model_artifacts.output_s3_path}")

# Save job ID for later
job_id = training_result.job_id
escrow_uri = training_result.model_artifacts.checkpoint_s3_path
output_path = training_result.model_artifacts.output_s3_path

## Step 7: Monitor Training Progress

### A) While training is ongoing

In [ ]:
# View recent training logs
print("📋 Training Logs:")
print("=" * 80)
customizer.get_logs(limit=50, start_from_head=False)

### B) After Training is completed

In [ ]:
job_id = training_result.job_id

In [ ]:
from amzn_nova_customization_sdk.monitor.log_monitor import CloudWatchLogMonitor

monitor = CloudWatchLogMonitor.from_job_id(job_id=job_id, platform=platform)
monitor.show_logs(limit=100, start_from_head=True)

## Step 8 Evaluate the custom Model (After Training Completes)

Evaluation jobs allow you to test your customized model against pre-set or custom benchmarks.

In [ ]:
# TODO: Update these values for your environment
S3_BUCKET = S3_BUCKET
S3_DATA_PATH = f"s3://{S3_BUCKET}/demo/input"
S3_OUTPUT_PATH = f"s3://{S3_BUCKET}/demo/output"

infra = SMTJRuntimeManager(
    instance_type="ml.p5.48xlarge",  # Change the instance type if needed (e.g. p5.48xlarge)
    instance_count=1,
)

evaluator = NovaModelCustomizer(
    model=Model.NOVA_LITE_2,  # You can also use your trained model here for eval
    method=TrainingMethod.EVALUATION,
    infra=infra,
    data_s3_path=S3_DATA_PATH,  # The data_s3_path is not used in eval job
    output_s3_path=S3_OUTPUT_PATH,  # This will be your eval output path
)

### Evaluation can be 3 dimensional 
- Using Public Benchmark to check on Models generalizability is maintained or not.
- Using your custom Data to validate models performance on YOUR tasks.
- Using LLM As Judge in domains where response quality is hard to evaluate.

In [ ]:
from amzn_nova_customization_sdk.recipe_config.eval_config import EvaluationTask

mmlu_eval_result = evaluator.evaluate(
    job_name="eval-test-mmlu",  # The job name you specified
    eval_task=EvaluationTask.MMLU,  # The eval task
)

byod_eval_result = evaluator.evaluate(
    job_name="eval-test-byod",
    eval_task=EvaluationTask.GEN_QA,
    data_s3_path="s3://<data-s3-bucket>/nova-customization/gen_qa.jsonl",  # TODO: Replace with your data path
    # model_path='s3://customer-escrow-<your-model-ckpt-bucket>/your-model-path/' # TODO: Replace with your model path
    overrides={"max_new_tokens": 2048},
)

# byom_eval_result = evaluator.evaluate(
#     job_name='eval-test-byom',
#     eval_task=EvaluationTask.GEN_QA,
#     data_s3_path='s3://<your-byom-dataset-bucket>/input/eval/byom/byom_data.jsonl', # TODO: Replace with your dataset
#     processor={
#         "lambda_arn": "arn:aws:lambda:<region>:<account>:function:<lambda>" # TODO: Your byom lambda
#     }
# )

# llm_judge_eval_result = evaluator.evaluate(
#     job_name='eval-test-llm-judge',
#     eval_task=EvaluationTask.LLM_JUDGE,
#     data_s3_path='s3://<your-llm-judge-dataset-bucket>/input/eval/llm_judge/llm_judge.jsonl' # TODO: Replace with your dataset
# )

In [ ]:
print("  📍 Bring Your Own Data Job ID: ", byod_eval_result.job_id)
print("  📂 Bring Your Own Data Output Path:", byod_eval_result.eval_output_path)
print("  📍 MMLU Job ID:", mmlu_eval_result.eval_output_path)
print("  📂 MMLU Output Path:", mmlu_eval_result.eval_output_path)

In [ ]:
# View recent training logs

print("📋 Evaluation Job Logs:")
print("=" * 80)
evaluator.get_logs(limit=50, start_from_head=False)

## Step 9: Deploy Your Model (After Training Completes)

Once training is complete, deploy your model to Amazon Bedrock.

In [ ]:
# Retrieve the model artifact path from the `s3_output_path`
import json
import tarfile
import tempfile

# Download and extract manifest from S3
s3 = boto3.client("s3")
bucket = S3_OUTPUT_PATH.split("/")[2]
key = f"{'/'.join(S3_OUTPUT_PATH.split('/')[3:])}/{job_id}/output/output.tar.gz"

print(f"Bucket is {bucket}, Key is {key}")

with tempfile.NamedTemporaryFile() as tmp_file:
    s3.download_file(bucket, key, tmp_file.name)
    with tarfile.open(tmp_file.name, "r:gz") as tar:
        manifest_content = tar.extractfile("manifest.json").read()
        manifest = json.loads(manifest_content)

model_artifacts_path = manifest["checkpoint_s3_bucket"]
print(model_artifacts_path)

In [ ]:
# Get the model artifacts path from training result
# After training completes, use:

# Deploy to Bedrock On-Demand
deployment_result = customizer.deploy(
    model_artifact_path=model_artifacts_path,
    deploy_platform=DeployPlatform.BEDROCK_OD,
    endpoint_name="my-custom-nova-model",
)

print("\n🚀 Model deployment started!")
print(f"   Endpoint Name: {deployment_result.endpoint.endpoint_name}")
print(f"   Status: {deployment_result.status}")

## Summary

You've completed the basic workflow:

✅ **Loaded and prepared data** in JSONL format  
✅ **Transformed data** to Nova's Converse format  
✅ **Configured runtime** infrastructure (SMTJ or HyperPod)  
✅ **Started training** with custom hyperparameters  
✅ **Monitored progress** via logs  
✅ **Deployed the model** to Amazon Bedrock  

## Next Steps

- **Use your own data**: Replace the sample data with your training dataset
- **Tune hyperparameters**: Adjust learning rate, batch size, epochs, etc.
- **Evaluate performance**: Use `customizer.evaluate()` to benchmark your model
- **Run batch inference**: Process large datasets with `customizer.batch_inference()`

## Available Models


## Available Training Methods

- `TrainingMethod.SFT_LORA` - Supervised Fine-Tuning with LoRA
- `TrainingMethod.SFT_FULL` - Full supervised fine-tuning
- `TrainingMethod.RFT_LORA` - Reinforcement Fine-Tuning with LoRA

## Resources

- [Amazon Nova Models](https://aws.amazon.com/bedrock/nova/)
- [SageMaker Documentation](https://docs.aws.amazon.com/sagemaker/)
- [Amazon Bedrock](https://aws.amazon.com/bedrock/)